In [1]:
with open("files/OldManAndSea.txt","r", encoding='utf-8-sig') as f:
    text=f.read()
# 加载原始文本并将其拆分为单个字符
text=list(text)
for i in range(len(text)):
    if text[i]=='"':
        if text[i+1]==' ' or text[i+1]=='\n':
            # 如果直引号后跟一个空格或换行符，则将其更改为 ” 引号
            text[i]='”'
        if text[i+1]!=' ' and text[i+1]!='\n':
            # 否则，将其更改为 “ 引号
            text[i]='“'    #C
    if text[i]=="'":
        if text[i-1]!=' ' and text[i-1]!='\n':
            # 将直单引号转换为 ’ 号
            text[i]='’'
# 将单个字符重新连接回文本
text="".join(text)

In [2]:
# 从第二本小说读取文本
with open("files/ToWhomTheBellTolls.txt","r", encoding='utf-8-sig') as f:
    text1=f.read()
# 从第三本小说读取文本
with open("files/FarewellToArms.txt","r", encoding='utf-8-sig') as f:
    text2=f.read()
# 合并三本小说的文本
text=text+" "+text1+" "+text2    #C

with open("files/ThreeNovels.txt","w", 
          encoding='utf-8-sig') as f:
    # 保存合并后的文本
    f.write(text)
print(text[:250])

He was an old man who fished alone in a skiff in the Gulf Stream and he
had gone eighty-four days now without taking a fish.  In the first
forty days a boy had been with him.  But after forty days without a
fish the boy’s parents had told him that th


In [3]:
# 将换行符替换为空格
text=text.lower().replace("\n", " ")
chars=set(text.lower())
# 识别所有标点符号
punctuations=[i for i in chars if i.isalpha()==False
              and i.isdigit()==False]
print(punctuations)

for x in punctuations:
    # 在标点符号周围插入空格
    text=text.replace(f"{x}", f" {x} ")
text_tokenized=text.split()
# 计算独特词元的数量
unique_tokens=set(text_tokenized)
print(len(unique_tokens))

['‘', '?', ';', ' ', '“', '.', ':', '-', '’', '!', ')', '&', '(', '”', ',']
10599


In [4]:
from collections import Counter   

word_counts=Counter(text_tokenized)    
words=sorted(word_counts, key=word_counts.get,
                      reverse=True)     
# 添加 UNK 词元
words.append("UNK")
text_length=len(text_tokenized)
# 计算词汇表的大小
ntokens=len(words)
print(f"the text contains {text_length} words")
print(f"there are {ntokens} unique tokens")
# 将词元映射到索引
word_to_int={v:k for k,v in enumerate(words)}
# 将索引映射到词元
int_to_word={v:k for k,v in word_to_int.items()}
print({k:v for k,v in word_to_int.items() if k in words[:10]})
print({k:v for k,v in int_to_word.items() if v in words[:10]})

the text contains 364463 words
there are 10600 unique tokens
{'.': 0, 'the': 1, ',': 2, '“': 3, '”': 4, 'and': 5, 'i': 6, 'he': 7, 'to': 8, 'it': 9}
{0: '.', 1: 'the', 2: ',', 3: '“', 4: '”', 5: 'and', 6: 'i', 7: 'he', 8: 'to', 9: 'it'}


In [5]:
print(text_tokenized[0:20])
wordidx=[word_to_int[w] for w in text_tokenized]  
print([word_to_int[w] for w in text_tokenized[0:20]])

['he', 'was', 'an', 'old', 'man', 'who', 'fished', 'alone', 'in', 'a', 'skiff', 'in', 'the', 'gulf', 'stream', 'and', 'he', 'had', 'gone', 'eighty']
[7, 14, 99, 93, 63, 85, 3818, 311, 15, 11, 657, 15, 1, 2369, 514, 5, 7, 24, 220, 2016]


In [6]:
import torch
# 将序列长度设置为128个索引
seq_len=128  
xys=[]
for n in range(0, len(wordidx)-seq_len-1):
    # 输入序列x包含训练文本中的128个连续索引
    x = wordidx[n:n+seq_len]
    # 将x向右移动一个位置，并将其作为输出y
    y = wordidx[n+1:n+seq_len+1]
    # 将(x, y)对添加到训练数据中
    xys.append((torch.tensor(x),(torch.tensor(y))))

In [8]:
from torch.utils.data import DataLoader

batch_size=32
loader = DataLoader(xys, batch_size=batch_size, shuffle=True)

x,y=next(iter(loader))
print(x)
print(y)
print(x.shape,y.shape)

tensor([[  37,    0,   25,  ...,   21,  122,  103],
        [9350,    1,  249,  ...,  336,  512,   53],
        [1388,   34,  136,  ...,   51,   12, 1844],
        ...,
        [   5,  200,   19,  ...,  180,   18,    4],
        [ 877,  199,    2,  ...,    4,    3,   71],
        [  34, 1797,    0,  ..., 1617,  584,    7]])
tensor([[   0,   25,  531,  ...,  122,  103,  575],
        [   1,  249,    5,  ...,  512,   53,   52],
        [  34,  136,  203,  ...,   12, 1844,   18],
        ...,
        [ 200,   19, 1757,  ...,   18,    4,   41],
        [ 199,    2,    4,  ...,    3,   71,   87],
        [1797,    0,    1,  ...,  584,    7, 1207]])
torch.Size([32, 128]) torch.Size([32, 128])


In [9]:
import torch
from torch import nn
import math

device="cuda" if torch.cuda.is_available() else "cpu"
class GELU(nn.Module):
    def forward(self, x):
        return 0.5*x*(1.0+torch.tanh(math.sqrt(2.0/math.pi)*\
                       (x + 0.044715 * torch.pow(x, 3.0))))

In [10]:
class Config():
    def __init__(self):
        self.n_layer = 3
        self.n_head = 4
        self.n_embd = 256
        self.vocab_size = ntokens
        self.block_size = 128 
        self.embd_pdrop = 0.1
        self.resid_pdrop = 0.1
        self.attn_pdrop = 0.1
        
config=Config()

In [11]:
import torch.nn.functional as F
class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.attn_dropout = nn.Dropout(config.attn_pdrop)
        self.resid_dropout = nn.Dropout(config.resid_pdrop)
        self.register_buffer("bias", torch.tril(torch.ones(\
                   config.block_size, config.block_size))
             .view(1, 1, config.block_size, config.block_size))
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        B, T, C = x.size() 
        q, k ,v  = self.c_attn(x).split(self.n_embd, dim=2)
        hs = C // self.n_head
        k = k.view(B, T, self.n_head, hs).transpose(1, 2) 
        q = q.view(B, T, self.n_head, hs).transpose(1, 2) 
        v = v.view(B, T, self.n_head, hs).transpose(1, 2) 

        att = (q @ k.transpose(-2, -1)) *\
            (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:,:,:T,:T] == 0, \
                              float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_dropout(att)
        y = att @ v 
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.resid_dropout(self.c_proj(y))
        return y

In [12]:
class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = nn.ModuleDict(dict(
            c_fc   = nn.Linear(config.n_embd, 4 * config.n_embd),
            c_proj = nn.Linear(4 * config.n_embd, config.n_embd),
            act    = GELU(),
            dropout = nn.Dropout(config.resid_pdrop),
        ))
        m = self.mlp
        self.mlpf=lambda x:m.dropout(m.c_proj(m.act(m.c_fc(x)))) 

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlpf(self.ln_2(x))
        return x

In [13]:
class Model(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.block_size = config.block_size
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.embd_pdrop),
            h = nn.ModuleList([Block(config) 
                               for _ in range(config.n_layer)]),   
            ln_f = nn.LayerNorm(config.n_embd),))
        self.lm_head = nn.Linear(config.n_embd,
                                 config.vocab_size, bias=False)      
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):    
                torch.nn.init.normal_(p, mean=0.0, 
                  std=0.02/math.sqrt(2 * config.n_layer))
    def forward(self, idx, targets=None):
        b, t = idx.size()
        pos = torch.arange(0,t,dtype=torch.long).unsqueeze(0).to(device)
        tok_emb = self.transformer.wte(idx) 
        pos_emb = self.transformer.wpe(pos) 
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)
        return logits

In [14]:
model=Model(config)
model.to(device)
num=sum(p.numel() for p in model.transformer.parameters())
print("number of parameters: %.2fM" % (num/1e6,))
print(model)

number of parameters: 5.12M
Model(
  (transformer): ModuleDict(
    (wte): Embedding(10600, 256)
    (wpe): Embedding(128, 256)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-2): 3 x Block(
        (ln_1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=256, out_features=768, bias=True)
          (c_proj): Linear(in_features=256, out_features=256, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (mlp): ModuleDict(
          (c_fc): Linear(in_features=256, out_features=1024, bias=True)
          (c_proj): Linear(in_features=1024, out_features=256, bias=True)
          (act): GELU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
 

In [15]:
lr=0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

In [16]:
model.train()  
for i in range(1,41):
    tloss = 0.
    # 遍历所有训练数据批次
    for idx, (x,y) in enumerate(loader):
        x,y=x.to(device),y.to(device)
        output = model(x)
        # 将模型预测与实际输出进行比较
        loss=loss_func(output.view(-1,output.size(-1)),
                           y.view(-1))
        optimizer.zero_grad()
        loss.backward()
        # 将梯度范数裁剪为1
        nn.utils.clip_grad_norm_(model.parameters(),1)
        # 调整模型参数
        optimizer.step()
        tloss += loss.item()
    print(f'epoch {i} loss {tloss/(idx+1)}') 
    if i%10==0:
        # 每隔 10 个 epoch 保存一次模型
        torch.save(model.state_dict(),f'files/GPTe{i}.pth') 

epoch 1 loss 3.7760729258076786
epoch 2 loss 2.676208010064096
epoch 3 loss 2.127361443472855
epoch 4 loss 1.7656190975592254
epoch 5 loss 1.5094296120077073
epoch 6 loss 1.3176188678852854
epoch 7 loss 1.16998387734186
epoch 8 loss 1.0534174176305664
epoch 9 loss 0.9589971623844952
epoch 10 loss 0.8814476710850045
epoch 11 loss 0.8167824102331711
epoch 12 loss 0.7622842388382993
epoch 13 loss 0.7160491116767648
epoch 14 loss 0.6771292377655271
epoch 15 loss 0.6431691947530699
epoch 16 loss 0.6140689558583321
epoch 17 loss 0.5882501356236947
epoch 18 loss 0.5654522757679512
epoch 19 loss 0.5448244017199265
epoch 20 loss 0.526271384592197
epoch 21 loss 0.5096928720396348
epoch 22 loss 0.4944293839759155
epoch 23 loss 0.48043519630439246
epoch 24 loss 0.46764937905902587
epoch 25 loss 0.45589859904837526
epoch 26 loss 0.4449365567589359
epoch 27 loss 0.43470808733814154
epoch 28 loss 0.4251141446390157
epoch 29 loss 0.41644868460393386
epoch 30 loss 0.4082125714096185
epoch 31 loss 0.400

In [26]:
def sample(idx, weights, max_new_tokens, temperature=1.0, top_k=None):
    model.eval()
    # 加载训练过的模型
    model.load_state_dict(torch.load(weights, weights_only=False))
    original_length=len(idx[0])
    # 生成固定数量的新索引
    for _ in range(max_new_tokens):
        if idx.size(1) <= config.block_size:
            idx_cond = idx  
        else:
            idx_cond = idx[:, -config.block_size:]
        # 使用模型进行预测
        logits = model(idx_cond.to(device))
        logits = logits[:, -1, :] / temperature
        if top_k is not None:
            v, _ = torch.topk(logits, top_k)
            logits[logits < v[:, [-1]]] = -float('Inf')
        probs = F.softmax(logits, dim=-1)
        idx_next=torch.multinomial(probs,num_samples=1)
        # 将新的索引添加到序列的末尾
        idx = torch.cat((idx, idx_next.cpu()), dim=1)
    # 仅输出新的索引
    return idx[:, original_length:]  

In [27]:
UNK=word_to_int["UNK"]
def generate(prompt, weights, max_new_tokens, temperature=1.0,
             top_k=None):
    # 确保提示并不为空
    assert len(prompt)>0, "prompt must contain at least one token"
    text=prompt.lower().replace("\n", " ")
    for x in punctuations:
        text=text.replace(f"{x}", f" {x} ")
    text_tokenized=text.split() 
    # 将提示转换为索引序列
    idx=[word_to_int.get(w,UNK) for w in text_tokenized]
    idx=torch.LongTensor(idx).unsqueeze(0)
    # 使用sample()函数生成新的索引
    idx=sample(idx, weights, max_new_tokens, temperature=1.0, top_k=None)
    # 将新的索引序列转换回文本
    tokens=[int_to_word[i] for i in idx.squeeze().numpy()] 
    text=" ".join(tokens)
    for x in '''”).:;!?,-‘’''':
        text=text.replace(f" {x}", f"{x}") 
    for x in '''“(-‘’''':
        text=text.replace(f"{x} ", f"{x}")     
    return prompt+" "+text

In [28]:
prompt="UNK"
for i in range(10):
    print(generate(prompt,'files/GPTe20.pth',max_new_tokens=20)[4:])
    print("-"*50)

shop. “all the poor hands and old fish,” he said. “but i was there
--------------------------------------------------
,” pablo said. “i do not like to have girls. i don’t care to
--------------------------------------------------
. it would be a great mistake to arrive at all regularly,” he said. “so long
--------------------------------------------------
completely and in the fall the cattle were cool. the lieutenant-colonel went in to be brought in
--------------------------------------------------
way. it was all right. this was what golz had talked about. the longer he was around
--------------------------------------------------
four o’clock in the morning at marseilles when the mistral was blowing. the major said he had
--------------------------------------------------
carefully through the woods at night. “do you know anything about being conquered and so you think it
--------------------------------------------------
chairs.” “in two places in this month the open town,” she sa

In [29]:
prompt="UNK"
print(generate(prompt,'files/GPTe10.pth',max_new_tokens=50)[4:])

?” “they’ll ring for us. we’ll have to be awfully careful. you’ll have to prove you.” “have another vermouth?” i asked. “no thank you. i have to make you any report if


In [30]:
prompt="the old man saw the shark near the"
for i in range(10):
    print(generate(prompt,'files/GPTe40.pth',max_new_tokens=20))
    print("-"*50)

the old man saw the shark near the stars and his strange light the sun. it was cold after the sun went down and the old man
--------------------------------------------------
the old man saw the shark near the break through the water that was too far away as he knew. then he swung the tiller across the
--------------------------------------------------
the old man saw the shark near the stick break in the darkness; strange light the sun shone on him. he twisted the blade and as
--------------------------------------------------
the old man saw the shark near the stars. he must be an awfully old man and he will beat me. first he wore no water
--------------------------------------------------
the old man saw the shark near the break through the fish. he still hung to his left eye and then he put the oblong of the
--------------------------------------------------
the old man saw the shark near the stars. it was an easy shot now and he just then he got the mast on his shoulder and
------

In [31]:
prompt="the old man saw the shark near the"
for i in range(10):
    print(generate(prompt,'files/GPTe20.pth',max_new_tokens=20,
                  temperature=0.9,top_k=50))
    print("-"*50)

the old man saw the shark near the boy that. i must know that the sun was down and not let us go out and stay there
--------------------------------------------------
the old man saw the shark near the other lions. he still hung to a place where two stretcher-bearers stood beside fernando against the wall
--------------------------------------------------
the old man saw the shark near the gulf weed that were out and he loved in the water, now that the old man could see his
--------------------------------------------------
the old man saw the shark near the last almost unheard hum that comes like the rope to a club of aficionados banded together into the club finito
--------------------------------------------------
the old man saw the shark near the other boats. the old man’s head was clear and cold and his old legs. he was
--------------------------------------------------
the old man saw the shark near the same movement of the other three. he had taken his knife to go down to where

In [32]:
prompt="the old man saw the shark near the"
print(generate(prompt,'files/GPTe40.pth',max_new_tokens=50,
                  temperature=0.95,top_k=100))

the old man saw the shark near the stars. he leaned over the side and punched at him. he hit only meat and the hide was set hard and he barely got the knife in. the blow hurt not only his hands but his shoulder too. but the shark came up fast with his
